In [17]:
%matplotlib inline
import numpy as np
from scipy.optimize import root_scalar, minimize_scalar
from math import log, ceil
import ipywidgets as widgets
import matplotlib.pyplot as plt

class StabilityCalculator:
    """Calculator used to carry out the analytic operations for a single instance of a PARSimple system.

    Supplied with the parameters for a system modelled by PARSimple, this class computes the trajectory
    of the system and determines information for outputs, such as the domains, crossover points, 
    time in danger. etc.

    Attributes:
        I (float): The initial inexperienced population.
        E (float): The initial experienced population.
        N (int): The number of non-operational positions.
        P (int): The number of flying positions.
        R (int): The number of hours required to upgrade.
        g (float): The OTU graduate rate.
        a (float): The attrition rate.
        h (float): The rate of other experienced losses.
        rI (float): The max inexperienced training rate.
        rE (float): The max experienced mentoring rate.
        rY (float): The max overall flying rate.
    """    
    
    def __init__(self, I, E, N, P, R, g, a, h, rI, rE, rY): 
        """Initializes the instance of the calculator.

        Args:
            See class attributes.
        """
        self.I=I
        self.E=E
        self.N=N
        self.P=P
        self.R=R
        self.g=g
        self.a=a
        self.h=h
        self.rI=rI
        self.rE=rE
        self.rY=rY
    
    def run(self):
        """Main method of the class used to run the analysis until success or failure.

        Returns:
            time_in_danger (float): The time spent in domains other than mentee-limited.
            status (str): Indicates the end state of the system and cause of crash (if applicable).
            exceeds_P (bool): Indicates a transient violation of PML.
            tracking_info (dict): Contains the information needed to track the system through its
                crossover points. The keys are the times of the crossover points, the domains, 
                I (at the crossover point) and E (at the crossover point). The values are lists.
        """
        
        fail_init_cond = False
        fail_ss = False
        if not self.check_passes_ss():
            fail_ss = True
        elif not self.check_passes_initial_condition():
            fail_init_cond = True

        current_domain = self.determine_domain();
        time_in_danger = 0
        exceeds_P = False
        current_t = 0
        tracking_info = {'domains':[],'xover_points':[],'I':[],'E':[]}
        done = False 
        while not done:
            temp_E = self.E
            temp_I = self.I
            tracking_info['E'].append(temp_E)
            tracking_info['I'].append(temp_I)
            tracking_info['xover_points'].append(current_t)
            tracking_info['domains'].append(current_domain)
            
            if current_domain == 'mentee':
                t1 = self.find_mentee_yfr_crossover()
                t2 = self.find_mentee_mentor_crossover()
                t3 = self.find_mentee_position_crossover()
                t = min((i for i in (t1,t2,t3) if i>0), default=-1)
                if t == -1:
                    # Last domain, check if it violates PML in the future
                    if not exceeds_P and self.violates_P('mentee', temp_I, temp_E, current_t, -1, True):
                        exceeds_P = True
                    done = True
                else:
                    self.E = self.E_mentee_lim(temp_E, temp_I, t)
                    self.I = self.I_mentee_lim(temp_E, temp_I, t)
                    if self.E < 0 or self.I < 0: # Do not move to next domain if population < 0
                        break
                    # Check for voilation of PML between current time and next crossover point
                    if not exceeds_P and self.violates_P('mentee', temp_I, temp_E, current_t, current_t+t, False):
                        exceeds_P = True
                    current_t += t
                    if t == t1:
                        current_domain = 'yfr'
                    elif t == t2:
                        current_domain = 'mentor'
                    else:
                        current_domain = 'position'
            elif current_domain == 'mentor':
                t1 = self.find_mentor_yfr_crossover()
                t2 = self.find_mentor_mentee_crossover()
                t3 = self.find_mentor_position_crossover()
                t = min((i for i in (t1,t2,t3) if i > 0), default=-1)
                if t == -1:
                    # Last domain, check if it violates PML in the future
                    if not exceeds_P and self.violates_P('mentor', temp_I, temp_E, current_t, -1, True):
                        exceeds_P = True
                    done = True
                    # Determine crash info if mentor-limited is the last domain
                    crashed_E_time = self.find_crash_mentor(self.E)
                    tracking_info['E'].append(0)
                    tracking_info['I'].append(self.I_mentor_lim(self.E, self.I, crashed_E_time))
                    tracking_info['xover_points'].append(crashed_E_time)
                    tracking_info['domains'].append('mentor')
                else:
                    self.E = self.E_mentor_lim(temp_E, temp_I, t)
                    self.I = self.I_mentor_lim(temp_E, temp_I, t)
                    # Do not move to next domain if population < 0.
                    # This also indicates a crash, add that info to tracking_info.
                    if self.E < 0 or self.I < 0: 
                        crashed_E_time = self.find_crash_mentor(temp_E)
                        tracking_info['E'].append(0)
                        tracking_info['I'].append(self.I_mentor_lim(temp_E, temp_I, crashed_E_time))
                        tracking_info['xover_points'].append(crashed_E_time)
                        tracking_info['domains'].append('mentor')
                        break
                    # Check for voilation of PML between current time and next crossover point
                    if not exceeds_P and self.violates_P('mentor', temp_I, temp_E, current_t, current_t+t, False):
                        exceeds_P = True
                    current_t += t
                    time_in_danger += t
                    if t == t1:
                        current_domain = 'yfr'
                    elif t == t2:
                        current_domain = 'mentee'
                    else:
                        current_domain = 'position'
            elif current_domain == 'yfr':
                t1 = self.find_yfr_mentee_crossover()
                t2 = self.find_yfr_mentor_crossover()
                t3 = self.find_yfr_position_crossover()
                t = min((i for i in (t1,t2,t3) if i > 0), default=-1)
                if t==-1:
                    # Last domain, check if it violates PML in the future
                    if not exceeds_P and self.violates_P('yfr', temp_I, temp_E, current_t, -1, True):
                        exceeds_P = True
                    done = True
                else:
                    self.E = self.E_yfr_lim(temp_E, t)
                    self.I = self.I_yfr_lim(temp_I, t)
                    if self.E < 0 or self.I < 0: # Do not move to next domain if population < 0.
                        break
                    # Check for voilation of PML between current time and next crossover point
                    if not exceeds_P and self.violates_P('yfr', temp_I, temp_E, current_t, current_t+t, False):
                        exceeds_P = True
                    current_t += t
                    time_in_danger += t
                    if t == t1:
                        current_domain = 'mentee' 
                    elif t == t2:
                        current_domain = 'mentor'
                    else:
                        current_domain = 'position'
            else:
                t1 = self.find_position_mentee_crossover()
                t2 = self.find_position_yfr_crossover()
                t3 = self.find_position_mentor_crossover()
                t = min((i for i in (t1,t2,t3) if i > 0), default=-1)
                if t==-1:
                    done = True
                    # If position-limited is the last domain, get crash info
                    crashed_E_time = self.find_crash_position(self.I)
                    tracking_info['E'].append(0)
                    tracking_info['I'].append(self.I_position_lim(self.I, crashed_E_time))
                    tracking_info['xover_points'].append(crashed_E_time)
                    tracking_info['domains'].append('position')
                else:
                    self.I = self.I_position_lim(temp_I, t)
                    self.E = self.E_position_lim(temp_E, temp_I, t)
                    if self.E < 0 or self.I < 0: # Do not move to next domain if population < 0.
                        break
                    current_t += t
                    time_in_danger += t
                    if t == t1:
                        current_domain = 'mentee' 
                    elif t == t2:
                        current_domain = 'yfr'
                    else:
                        current_domain = 'mentor'

        if fail_init_cond:
            return 500, 'FAIL_IC', False, tracking_info
        elif fail_ss:
            return 500, 'FAIL_SS', False, tracking_info
        if current_domain == 'mentee':
            return time_in_danger, 'PASS', exceeds_P, tracking_info
        elif current_domain == 'yfr':
            return 500, 'FAIL_YFR', exceeds_P, tracking_info
        elif current_domain == 'mentor':
            return 500, 'FAIL_MENTOR', exceeds_P, tracking_info
        else:
            return 500, 'FAIL_POSITION', exceeds_P, tracking_info
        
                
    def check_passes_ss(self):
        """Returns whether or not the system passes the steady-state tests."""
        try:
            Iss = self.g*self.R/self.rI
            Ess = (self.g-self.h-self.a*self.N)/self.a
            # Rounding to prevent float comparison errors
            rounded_rIIss = round(self.rI*Iss, 10)
            rounded_rEEss = round(self.rE*Ess, 10)
            rounded_rY = round(self.rY, 10)
            return Iss>0 and Ess>0 and Iss+Ess<=self.P and (rounded_rEEss>=rounded_rIIss or rounded_rEEss>=rounded_rY) and\
                (rounded_rY>=rounded_rIIss or rounded_rY>=rounded_rEEss)
        except:
            return False

    def check_passes_initial_condition(self):
        """Returns whether or not the system passes the initial condition test."""
        
        # Rounding to prevent float comparison errors
        rounded_rII = round(self.rI*self.I, 10)
        rounded_rEE = round(self.rE*self.E, 10)
        rounded_rY = round(self.rY, 10)
        rounded_E = round(self.E, 10)
        rounded_crit = round(((self.h+self.a*self.N)/(self.rE/self.R-self.a)), 10)
        return rounded_rEE>rounded_rII or rounded_rEE>rounded_rY or rounded_E>=rounded_crit
        
    def determine_domain(self):
        """Returns the domain of the system in the current state as a string."""
        current_min = min(self.rI*self.I, self.rE*self.E, self.rY, self.rE*(self.P-self.I))

        # If tied, the system is mentor-limited if E is decreasing, mentee-limited otherwise.
        if self.rI*self.I == current_min and self.rE*self.E == current_min:
            if self.rE/self.R*self.E-self.h-self.a*self.E-self.a*self.N < 0:
                return 'mentor'
            else:
                return 'mentee'
        # If tied, the system is position-limited if I is increasing, mentor-limited otherwise.
        elif self.rE*self.E == current_min and self.rE*(self.P-self.I) == current_min:
            if self.g-self.rE*self.E/self.R > 0:
                return 'position'
            else:
                return 'mentor'
        elif self.rI*self.I == current_min:
            return 'mentee'
        elif self.rE*self.E == current_min:
            return 'mentor'
        elif self.rY == current_min:
            return 'yfr'
        else:
            return 'position'
        

    def find_yfr_mentee_crossover(self):
        """Returns the crossover point from YFR-limited to mentee-limited."""
        try:
            return (self.rY/self.rI-self.I)*(self.R/(self.g-self.rY))
        except:
            return -1

    def find_yfr_mentor_crossover(self):
        """Returns the crossover point from YFR-limited to mentor-limited."""
        try:
            k=-self.rY+self.h*self.R+self.a*self.N*self.R
            log_arg = (self.rY*self.a*self.R+self.rE*self.k)/(self.rE*(self.E*self.a*self.R+k))
            return -log(log_arg)/self.a
        except:
            return -1

    def find_yfr_position_crossover(self):
        """Returns the crossover point from YFR-limited to position-limited."""
        k1 = self.I-self.P+self.g*self.R/self.rE
        try:
            return (self.rE*self.P-self.rY-self.rE*self.I)/(self.rE*(self.g-self.rY/self.R))
        except:
            return -1

    def find_mentee_yfr_crossover(self):
        """Returns the crossover point from mentee-limited to YFR-limited."""
        try:
            k1=self.I-self.g*self.R/self.rI
            log_arg = (self.rY-self.g*self.R)/(self.rI*self.k1)
            return -self.R*log(log_arg)/self.rI
        except:
            return -1

    def find_mentee_mentor_crossover(self):
        """Returns the crossover point from mentee-limited to mentor-limited."""
        try:
            func = lambda t: self.rI*self.I_mentee_lim(self.E, self.I, t)- self.rE*self.E_mentee_lim(self.E, self.I, t)
            # To use root_scalar, we need a bracket that evaluates to opposite signs at each end.
            t_sol = root_scalar(func, bracket=[0.01, self.get_bracket_upper(func)])
            return t_sol.root
        except:
            return -1

    def find_mentee_position_crossover(self):
        """Returns the crossover point from mentee-limited to position-limited."""
        try:
            k1=self.I-self.g*self.R/self.rI
            log_arg = (self.rE*self.P/(self.rI+self.rE)-self.g*self.R/self.rI)/k1
            return -self.R/self.rI*log(log_arg)
        except:
            return -1

    def find_mentor_yfr_crossover(self):
        """Returns the crossover point from mentor-limited to YFR-limited."""
        k1=self.E-(self.h+self.a*self.N)/(self.rE/self.R-self.a)
        k2=self.I+k1/(1-self.a*self.R/self.rE)
        try:
            log_arg = (self.rY-(self.rE*(self.h+self.a*self.N))/(self.rE/self.R-self.a))/(self.rE*k1)
            return log(log_arg)/(self.rE/self.R-self.a)
        except:
            return -1

    def find_mentor_mentee_crossover(self):
        """Returns the crossover point from mentor-limited to mentee-limited."""
        try:
            func = lambda t: self.rE*self.E_mentor_lim(self.E, self.I, t)-self.rI*self.I_mentor_lim(self.E, self.I, t)
            # To use root_scalar, we need a bracket that evaluates to opposite signs at each end.
            t_sol = root_scalar(func, bracket=[0.01, self.get_bracket_upper(func)])
            return t_sol.root
        except:
            return -1

    def find_mentor_position_crossover(self):
        """Returns the crossover point from mentor-limited to position-limited."""
        try:
            func = lambda t: self.E_mentor_lim(self.E, self.I, t)+self.I_mentor_lim(self.E, self.I, t)-self.P
            # To use root_scalar, we need a bracket that evaluates to opposite signs at each end.
            t_sol = root_scalar(func, bracket=[0.01, self.get_bracket_upper(func)])
            return t_sol.root
        except:
            return -1
            
    def find_position_mentee_crossover(self):
        """Returns the crossover point from position-limited to mentee-limited."""
        k1 = self.I-self.P+self.g*self.R/self.rE
        try:
            log_arg = (self.g*self.R*(1+self.rI/self.rE)-self.rI*self.P)/(k1*(self.rI+self.rE))
            return self.R/self.rE*log(log_arg)
        except:
            return -1

    def find_position_mentor_crossover(self):
        """Returns the crossover point from position-limited to mentor-limited."""
        try:
            func = lambda t: self.P-self.E_position_lim(self.E, self.I, t)-self.I_position_lim(self.I, t)
            # To use root_scalar, we need a bracket that evaluates to opposite signs at each end.
            t_sol = root_scalar(func, bracket=[0.01, self.get_bracket_upper(func)])
            return t_sol.root
        except:
            return -1
    
    def find_position_yfr_crossover(self):
        """Returns the crossover point from position-limited to YFR-limited."""
        k1 = self.I-self.P+self.g*self.R/self.rE
        try:
            log_arg = (self.g*self.R-self.rY)/(self.rE*k1)
            return self.R/self.rE*log(log_arg)
        except:
            return -1   
    
    def get_bracket_upper(self, func):
        """Returns a value of t to use as an upper-bound on the bracket for the root_scalar function.

        Args:
            func (function): The function for which a bracket value needs to be found.
        """
        i = 1
        done = False
        # Checking for sign change within next 40 years. The functions will all initially
        # be negative (by construction), so we are looking for a positive value.
        while i < 2080 and done == False:
            if func(i) > 0:
                done = True
            i+=1
        return i
    
    def E_mentor_lim(self, E, I, t):
        """Returns the value of E in the mentor-limited domain.

        Args:
            E (float): The initial value of E.
            I (float): The initial value of I.
            t (float): The time at which to calculate the new value of E.
        """
        try:
            k1=E-(self.h+self.a*self.N)/(self.rE/self.R-self.a)
            k2=I+k1/(1-self.a*self.R/self.rE)
            return k1*np.exp((self.rE/self.R-self.a)*t)+(self.h+self.a*self.N)/(self.rE/self.R-self.a)
        except ZeroDivisionError:
            return 10000000
    
    def I_mentor_lim(self, E, I, t):
        """Returns the value of I in the mentor-limited domain.

        Args:
            E (float): The initial value of E.
            I (float): The initial value of I.
            t (float): The time at which to calculate the new value of I.
        """
        try:
            k1=E-(self.h+self.a*self.N)/(self.rE/self.R-self.a)
            k2=I+k1/(1-self.a*self.R/self.rE)
            return -k1/(1-self.a*self.R/self.rE)*np.exp((self.rE/self.R-self.a)*t)+\
                    ((-self.h-self.a*self.N)/(1-self.a*self.R/self.rE)+self.g)*t+k2
        except ZeroDivisionError:
            return 10000000
    
    def E_mentee_lim(self, E, I, t):
        """Returns the value of E in the mentee-limited domain.

        Args:
            E (float): The initial value of E.
            I (float): The initial value of I.
            t (float): The time at which to calculate the new value of E.
        """
        try:
            k1=I-self.g*self.R/self.rI
            k2=E+(self.rI/(self.rI-self.a*self.R))*k1-(self.g-self.h-self.a*self.N)/self.a
            return -self.rI/(self.rI-self.a*self.R)*k1*np.exp(-self.rI*t/self.R)+k2*np.exp(-self.a*t)+\
                    (self.g-self.h-self.a*self.N)/self.a
        except ZeroDivisionError:
            return 10000000
    
    def I_mentee_lim(self, E, I, t):
        """Returns the value of I in the mentee-limited domain.

        Args:
            E (float): The initial value of E.
            I (float): The initial value of I.
            t (float): The time at which to calculate the new value of I.
        """
        try:
            k1=I-self.g*self.R/self.rI
            k2=E+(self.rI/(self.rI-self.a*self.R))*k1-(self.g-self.h-self.a*self.N)/self.a
            return k1*np.exp(-self.rI*t/self.R)+self.g*self.R/self.rI
        except ZeroDivisionError:
            return 10000000
    
    def E_yfr_lim(self, E, t):
        """Returns the value of E in the YFR-limited domain.

        Args:
            E (float): The initial value of E.
            I (float): The initial value of I.
            t (float): The time at which to calculate the new value of E.
        """
        try:
            k=-self.rY+self.h*self.R+self.a*self.N*self.R
            return ((E*self.a*self.R+k)*np.exp(-self.a*t)-k)/(self.a*self.R)
        except ZeroDivisionError:
            return 10000000
    
    def I_yfr_lim(self, I, t):
        """Returns the value of I in the YFR-limited domain.

        Args:
            E (float): The initial value of E.
            I (float): The initial value of I.
            t (float): The time at which to calculate the new value of I.
        """
        try:
            return I+(self.g-self.rY/self.R)*t
        except ZeroDivisionError:
            return 10000000

    def I_position_lim(self, I, t):
        """Returns the value of I in the position-limited domain.

        Args:
            E (float): The initial value of E.
            t (float): The time at which to calculate the new value of I.
        """
        try:
            k1 = I-self.P+self.g*self.R/self.rE
            return k1*np.exp(self.rE/self.R*t)+self.P-self.g*self.R/self.rE
        except ZeroDivisionError:
            return 10000000

    def E_position_lim(self, E, I, t):
        """Returns the value of E in the position-limited domain.

        Args:
            E (float): The initial value of E.
            I (float): The initial value of I.
            t (float): The time at which to calculate the new value of E.
        """
        try:
            k1 = I-self.P+self.g*self.R/self.rE
            k2 = E+k1/(1+self.a*self.R/self.rE)-(self.g-self.h-self.a*self.N)/self.a
            return -k1/(1+self.a*self.R/self.rE)*np.exp(self.rE/self.R*t)+k2*np.exp(-self.a*t)+(self.g-self.h-self.a*self.N)/self.a
        except ZeroDivisionError:
            return 10000000

    def violates_P(self, domain, I, E, lower, upper, end):
        """Returns whether the system will violate PML within a timeframe.

        There is a transient violation of PML if P-I-E < 0 for some t in [t1,t2].
        Therefore, we can just check that the minimum of P-I-E < 0.
        
        Args:
            I (float): The initial value of I.
            E (float): The initial value of E.
            lower (float): The start time.
            upper (float): The end time.
            end (bool): Whether we are looking for a violation in the end state or not.
        """
        if domain == 'mentee':
            func = lambda t: self.P-self.I_mentee_lim(E, I, t)-self.E_mentee_lim(E, I, t)
        elif domain == 'mentor':
            func = lambda t: self.P-self.I_mentor_lim(E, I, t)-self.E_mentor_lim(E, I, t)
        else: # YFR-limited. No need to check position-limited as it violates by definition.
            func = lambda t: self.P-self.I_yfr_lim(I, t)-self.E_yfr_lim(E, t)

        if end:
            i = lower
            violates = False
            # Checking for sign change within next 40 years
            while i < lower+2080 and violates == False:
                if func(i) < 0:
                    violates = True
                i+=1
            return violates
        else:
            minimum = minimize_scalar(func, bounds=(lower,upper)).fun
            return minimum < 0;

    def find_crash_mentor(self, E):
        """Returns the time at which E will crash in the mentor-limited domain.

        Args:
            E (float): The current value of E.
        """
        try:
            k1=E-(self.h+self.a*self.N)/(self.rE/self.R-self.a)
            return log(-(self.h+self.a*self.N)/(k1*(self.rE/self.R-self.a)))/(self.rE/self.R-self.a)
        except:
            return -1

    def find_crash_position(self, I):
        """Returns the time at which E will crash in the mentor-limited domain.

        The system crashes if E=0, but equivalently if I=P.
        
        Args:
            I (float): The current value of I.
        """
        try:
            k1 = I-self.P+self.g*self.R/self.rE
            return self.R/self.rE*log(self.g*self.R/(k1*self.rE))
        except:
            return -1
        
    

In [15]:
def calc_u(I, E, P, R, rI, rE, rY):
    """Returns the upgrade rate.

    Args:
        See StabilityCalculator args.
    """
    current_min = min(rI*I, rE*E, rY, rE*(P-I))
    if rI*I == current_min:
        return rI/R*I
    elif rE*E == current_min:
        return rE/R*E
    elif rY/R == current_min:
        return rY/R
    else:
        return rE/R*(P-I)

def get_vector_coordinates(I, E, N, P, R, g, a, h, rI, rE, rY):
    """Returns the derivatives of I and E.

    Args:
        See StabilityCalculator args.
    """
    u = calc_u(I, E, P, R, rI, rE, rY)
    x = g-u
    y = u-h-a*E-a*N
    return x, y

# PARSimple "What If?" Analytic Tool
Welcome to the PARSimple "What If?" Analytic Tool. This tool allows you to modify the parameters to PARSimple and observe several outputs, such as areas of system sustainability, areas of transient voilation of PML, times in danger zones and the tracking of a specific system. The following table describes the variables used in this tool.

| Variable | Description                                         |
|----------|-----------------------------------------------------|
| I        | Initial inexperienced population                    |
| E        | Initial experienced population                      |
| N        | Non-operational positions                           |
| P        | Number of flying positions                          |
| R        | Hours required to upgrade                           |
| g        | OTU graduate rate (people/year)                     |
| h        | Other experienced population losses (people/year)   |
| a        | Attrition rate (/year)                              |
| rI       | Max inexperienced training rate (hours/year/mentee) |
| rE       | Max experienced mentoring rate (hours/year/mentee)  |
| rY       | rY: Max overall flying rate (YFR, hours/ys pressed.**

### Important Notes
- Note that if you are inputting the parameters with the keyboard instead of using the slider, you **MUST** press ENTER after inputting each parameter to ensure that the change will be reflected. If done successfully, the slider should move once ENTER is pressed.**
- The execution of an output generation instance is finished once the *Run Interact* button returns to being clickable. This is important to notice, as it is possible that in certain circumstance, changing some of the variables will not affect the output.

***



### Important Metrics Based on Initial Experienced and Inexperienced Pilots

This will generate the 5 plots of I vs E with differing information.
1. The final system sustainability.
2. The change in I and E as a vector field, colour-coded to also show the final system sustainability.
3. The times in danger zones (danger zones are defined as any domain other than mentee-limited).
4. The initial domains of the system.
5. The areas where the system will violate PML at some point during execution.

In [54]:
%matplotlib inline
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from matplotlib.colors import ListedColormap, Normalize


def plot_outputs(N=25, P=30, R=250, g=4, a=0.07,
                     h=1, rI=144, rE=144, rY=10000):
    """Function used to plot the main outputs of the analytic tool.

    Args:
        See StabilityCalculator args.
    """
    fig, ((ax0,ax1),(ax2,ax3),(ax4,ax5)) = plt.subplots(3,2, figsize=(20,30))

    x,y = np.meshgrid(np.linspace(0,P+1,P+1),np.linspace(0,P+1,P+1)) 
    u = np.zeros(x.shape) # dI/dt for vector field
    v = np.zeros(y.shape) # dE/dt for vector field
    times = np.zeros([P+1,P+1]) # Times in danger
    color_vals = np.zeros([P+1,P+1]) # Color values for end state and vector field
    color_vals_2 = np.zeros([P+1,P+1]) # Color values for transient PML violation
    color_vals_init_domain = np.zeros([P+1,P+1]) # Color values for initial domain plot
    
    for e in range(0, P+1):
        for i in range(0, P+1):
            # Update vector field
            x_val, y_val = get_vector_coordinates(i, e, N, P, R, g/52, a/52, h/52, rI/52, rE/52, rY/52)
            u[e,i] = x_val
            v[e,i] = y_val
            
            calculator = StabilityCalculator(i, e, N, P, R, g/52, a/52, h/52, rI/52, rE/52, rY/52)
            init_domain = calculator.determine_domain()
            time_in_danger, status, exceeds_P, _ = calculator.run()

            times[e,i] = time_in_danger

            # Update colors for end state and vector field
            c_value = None
            if status == 'PASS':
                c_value = 1
            elif status == 'FAIL_SS':
                c_value = 2
            elif status == 'FAIL_IC':
                c_value = 3
            elif status == 'FAIL_MENTOR':
                c_value = 4
            elif status == 'FAIL_POSITION':
                c_value = 5
            else: #FAIL_YFR
                c_value = 6
            color_vals[e,i] = c_value

            # Update colors for PML violation
            c_value_2 = None
            if status != 'PASS':
                c_value_2 = 3
            else:
                if not exceeds_P:
                    c_value_2 = 1
                else:
                    c_value_2 = 2
            color_vals_2[e,i] = c_value_2

            # Update colors for init domain plot
            c_value_dom = None
            if init_domain == 'mentee':
                c_value_dom = 0
            elif init_domain == 'mentor':
                c_value_dom = 1
            elif init_domain == 'yfr':
                c_value_dom = 2
            else:
                c_value_dom = 3
            color_vals_init_domain[e,i] = c_value_dom
                
    try:
        iss = g*R/rI
        ess = (g-h-a*N)/a
    except:
        iss = 0
        ess= 0

    # Plot steady-state
    ax0.plot(iss,ess,'k*', markersize=10) 
    ax1.plot(iss,ess,'k*', markersize=10)
    ax2.plot(iss,ess,'k*', markersize=10)
    ax3.plot(iss,ess,'k*', markersize=10)
    ax4.plot(iss,ess,'k*', markersize=10)
    # Star for marker on legend
    star = mlines.Line2D([], [], color='black', marker='*', linestyle='None',
                          markersize=10, label='Steady State')
    # Handle end state subplot
    values = [0,1,2,3,4,5,6,7]
    cmp = ListedColormap(['white','limegreen', 'lightslategray', 'chocolate', 'royalblue', 'crimson', 'magenta', 'black'])
    im = ax0.imshow(color_vals, origin='lower', cmap=cmp, vmin=0, vmax=7)
    colors = [im.cmap(value) for value in values]
    dashed_line = ax0.axline((0,P), slope=-1, linestyle='dashed', label='E+I=P', c='black')
    patches = [ mpatches.Patch(color=colors[1], label="Success"),
            mpatches.Patch(color=colors[2], label="Steady State Failure"),
            mpatches.Patch(color=colors[3], label="Initial Condition Failure"),
            mpatches.Patch(color=colors[4], label="Mentor-Limited Crash"),
            mpatches.Patch(color=colors[5], label="Position-Limited Crash"),
            mpatches.Patch(color=colors[6], label="YFR-Limited End State"),
            star,
              dashed_line]
    ax0.legend(handles=patches, loc='upper right')
    ax0.set_xticks(np.arange(-.5, P+0.5, 1), minor=True)
    ax0.set_yticks(np.arange(-.5, P+0.5, 1), minor=True)
    ax0.grid(which='minor')
    ax0.set_title('Stability Based on Initial Experienced and Inexperienced Pilots')
    ax0.set_xlabel('I0')
    ax0.set_ylabel('E0')

    # Handle vector field subplot
    patches = [ mpatches.Patch(color=colors[1], label="Success"),
            mpatches.Patch(color=colors[2], label="Steady State Failure"),
            mpatches.Patch(color=colors[3], label="Initial Condition Failure"),
            mpatches.Patch(color=colors[4], label="Mentor-Limited Crash"),
            mpatches.Patch(color=colors[5], label="Position-Limited Crash"),
            mpatches.Patch(color=colors[6], label="YFR-Limited End State"),
            star]
    ax1.legend(handles=patches, loc='upper right')
    ax1.quiver(x,y,u,v, color_vals, cmap=cmp, norm=Normalize(vmin=0, vmax=7))
    #ax1.set_title('Change in Pilot Populations')
    ax1.set_xlabel('I0')
    ax1.set_ylabel('E0')

    # Handle time in danger plot
    danger_hm = ax2.imshow(times, cmap='tab10', origin='lower', interpolation='bilinear')
    # Scale colorbar to be about the same height as plot
    hm_ratio = times.shape[0]/times.shape[1]
    fig.colorbar(danger_hm, ax=ax2, fraction=0.047*hm_ratio)
    ax2.axline((0,P), slope=-1, linestyle='dashed', label='E+I=P', c='black')
    #triangle1 = plt.Polygon([[-0.5,P+0.5],[P+0.5,P+0.5],[P+0.5,-0.5]], color='white')
    #ax2.add_patch(triangle1)
    ax2.set_xticks(np.arange(-.5, P+0.5, 1), minor=True)
    ax2.set_yticks(np.arange(-.5, P+0.5, 1), minor=True)
    ax2.grid(which='minor')
    ax2.set_title('Times in Danger Zones \n(in weeks, 500 indicates a failed instance)')
    ax2.set_xlabel('I0')
    ax2.set_ylabel('E0')

    # Handle initial domain subplot
    values = [0,1,2,3]
    cmp = ListedColormap(['limegreen', 'lightslategray', 'chocolate', 'royalblue'])
    im = ax3.imshow(color_vals_init_domain, origin='lower', cmap=cmp, vmin=0, vmax=3)
    colors = [ im.cmap(value) for value in values]
    patches = [ mpatches.Patch(color=colors[0], label="Mentee-Limited"),
            mpatches.Patch(color=colors[1], label="Mentor-Limited"),
            mpatches.Patch(color=colors[2], label="YFR-Limited"),
            mpatches.Patch(color=colors[3], label="Position-Limited"),
            star]
    ax3.legend(handles=patches, loc='upper right')
    ax3.set_xticks(np.arange(-.5, P+0.5, 1), minor=True)
    ax3.set_yticks(np.arange(-.5, P+0.5, 1), minor=True)
    ax3.grid(which='minor')
    #ax3.set_title('Initial Domain')
    ax3.set_xlabel('I0')
    ax3.set_ylabel('E0')

    # Handle PML violation subplot
    values = [0,1,2,3]
    cmp = ListedColormap(['white','forestgreen', 'tomato', 'saddlebrown'])
    im = ax4.imshow(color_vals_2, origin='lower', cmap=cmp, vmin=0, vmax=3)
    colors = [im.cmap(value) for value in values]
    ax4.axline((0,P), slope=-1, linestyle='dashed', label='E+I=P', c='black')
    patches = [mpatches.Patch(color=colors[1], label="Does Not Exceed P"),
                mpatches.Patch(color=colors[2], label="Exceeds P"),
                mpatches.Patch(color=colors[3], label="Failure"),
                star,
                dashed_line]
    ax4.legend(handles=patches, loc='upper right')
    ax4.set_title('Transient Violation of P')
    ax4.set_xlabel('I0')
    ax4.set_ylabel('E0')
    ax4.set_xticks(np.arange(-.5, 30.5, 1), minor=True)
    ax4.set_yticks(np.arange(-.5, 30.5, 1), minor=True)
    ax4.grid(which='minor')

    # Disable last plot
    ax5.axis('off')

# Enable to slider interaction
widgets.interact_manual(plot_outputs, N=widgets.IntSlider(value=25, min=0, max=100),
               P=widgets.IntSlider(value=30, min=0, max=100),
               R=widgets.IntSlider(value=250, min=0, max=1000),
               g=widgets.IntSlider(value=4, min=0, max=100),
               a=widgets.FloatSlider(value=0.07, min=0, max=1, step=0.01),
               h=widgets.IntSlider(value=1, min=0, max=100),
               rI=widgets.IntSlider(value=144, min=0, max=1000),
               rE=widgets.IntSlider(value=144, min=0, max=1000),
               rY=widgets.IntSlider(value=10000, min=0, max=10000))
pass

interactive(children=(IntSlider(value=25, description='N'), IntSlider(value=30, description='P'), IntSlider(va…

### Population Level Tracker

Supplied with all parameters to a system, this will generate a plot that tracks the system, showing the crossover points from one domain to another and the times at which these domain crossovers happen. It also shows when the system will crash or reach steady state.

In [50]:
%matplotlib inline
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

def steady_state_time(Iss, I, Ess, E, N, R, g, a, h, rI, last_t):
    """Returns the time at which the system will reach steady state.

    In theory, this never happens. However, for in practice, we will consider
    that the system reaches steady state when |Iss-I| < 1 and |Ess-E| < 1.

    Args:
        Iss (float): The mentee population steady-state value.
        Ess (float): The mentor population steady-state value.
        last_t (float): The time of the last crossover point.
        Others: See StabilityCalculator args.
    """
    k1 = I-Iss
    k2=E+(rI/(rI-a*R))*k1-(g-h-a*N)/a
    t1 = -1
    t2 = -1
    
    if abs(Iss-I)<1:
        t1 = 0
    elif Iss > I:
        t1 = -R*log(-1/k1)/rI
    else:
        t1 = -R*log(1/k1)/rI
        
    if abs(Ess-E)<1:
        t2 = 0
    elif Ess > E:
        func = lambda t: Ess-(-rI/(rI-a*R)*k1*np.exp(-rI*t/R)+k2*np.exp(-a*t)+\
                (g-h-a*N)/a)-1
        t2 = root_scalar(func, bracket=[0, get_threshold(func, last_t)]).root
    else:
        func = lambda t: (-rI/(rI-a*R)*k1*np.exp(-rI*t/R)+k2*np.exp(-a*t)+\
                (g-h-a*N)/a)-1-Ess
        t2 = root_scalar(func, bracket=[0, get_threshold(func, last_t)]).root
    
    return max(t1,t2)

def get_threshold(func, last_t):
        """Returns an upper bracket for use in steady_state_time().

        Calculate when the function will change signs. In our case,
        it is used to find when Ess-E-1 will change signs.
        """
        i = last_t
        done = False
        # Checking for sign change within next 40 years
        while i < last_t+10000 and done == False:
            if func(i) < 0:
                done = True
            i+=1
        return i
    

def plot_outputs(I=10, E=10, P=30, N=25, R=250, g=4,
                     a=0.07, h=1, rI=144, rE=144, rY=10000):
    """Plots the trajectory of the system.

    Args:
        See StabilityCalculator args.
    """
    
    plt.rcParams["figure.figsize"] = (8,5)

    calculator = StabilityCalculator(I, E, N, P, R, g/52, a/52, h/52, rI/52, rE/52, rY/52)
    _, status, _, tracking_info = calculator.run()
    I_points = tracking_info['I']
    E_points = tracking_info['E']
    domains = tracking_info['domains']
    xover_points = tracking_info['xover_points']

    try:
        iss = g/52*R/(rI/52)
        ess = (g/52-h/52-a/52*N)/(a/52)
    except:
        iss = 0
        ess= 0
        
    plt.plot(iss,ess,'g*', markersize=10)
    # Plot the points and arrow
    for x in range(len(I_points)):
        if domains[x] == 'mentee':
            marker = 'go'
        elif domains[x] == 'mentor':
            marker = 'ro'
        elif domains[x] == 'yfr':
            marker = 'bo'
        else:
            marker = 'yo'
        plt.plot(I_points[x],E_points[x],marker, markersize=5)
        if x == len(I_points)-1:
            break
        plt.arrow(I_points[x], E_points[x], I_points[x+1]-I_points[x], E_points[x+1]-E_points[x], width=0.15, length_includes_head=True,color='gray')
        
    # Plot arrow to steady state
    if tracking_info['domains'][-1] == 'mentee':
        plt.arrow(I_points[-1], E_points[-1], iss-I_points[-1], ess-E_points[-1], width=0.15, length_includes_head=True, color='gray')

    # Generate the table
    col_labels = ['Time (years)', 'Domain']
    table_vals = []
    cell_colors = []
    for i in range(len(xover_points)):
        table_vals.append([round(xover_points[i]/52,2),domains[i]+'-limited'])
        if domains[i] == 'mentee':
            cell_colors.append(['lightgreen','lightgreen'])
        elif domains[i] == 'mentor':
            cell_colors.append(['lightcoral','lightcoral'])
        elif domains[i] == 'yfr':
            cell_colors.append(['skyblue','skyblue'])
        else:
            cell_colors.append(['yellow','yellow'])
    if tracking_info['domains'][-1] == 'mentee':
        ss_time = steady_state_time(iss, I_points[-1], ess, E_points[-1], N, R, g/52, a/52, h/52, rI/52, xover_points[-1])+xover_points[-1]
        table_vals.append([round(ss_time/52,2), 'steady state'])
        cell_colors.append(['lightgreen','lightgreen'])
    elif tracking_info['domains'][-1] == 'mentor' or tracking_info['domains'][-1] == 'position':
        table_vals[-1][1] = 'crash'
       
    plt.table(cellText=table_vals,
                colLabels=col_labels,
                colWidths=[0.2,0.2],
                loc='upper right',
                cellLoc='left',
                cellColours = cell_colors)

    #plt.title('Trajectory of System')
    plt.xlabel('I')
    plt.ylabel('E')
    plt.axis([0, P, 0, P])

# Enable widget use
widgets.interact_manual(plot_outputs, I=widgets.IntSlider(value=18, min=0, max=100),
               E=widgets.IntSlider(value=8, min=0, max=100),
               P=widgets.IntSlider(value=30, min=0, max=100),
               N=widgets.IntSlider(value=25, min=0, max=100),
               R=widgets.IntSlider(value=250, min=0, max=1000),
               g=widgets.IntSlider(value=4, min=0, max=100),
               a=widgets.FloatSlider(value=0.07, min=0, max=1, step=0.01),
               h=widgets.IntSlider(value=1, min=0, max=100),
               rI=widgets.IntSlider(value=144, min=0, max=1000),
               rE=widgets.IntSlider(value=144, min=0, max=1000),
               rY=widgets.IntSlider(value=10000, min=0, max=100000))
pass

interactive(children=(IntSlider(value=18, description='I'), IntSlider(value=8, description='E'), IntSlider(val…

### Stability Based on Initial Experienced Pilots and Attrition Rate

This plots the final sustainability of the system with varying values of E and a.

In [44]:
%matplotlib inline
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from matplotlib.colors import ListedColormap


def plot_outputs(I=10, N=25, P=30, R=250, g=4,
                     h=1, rI=144, rE=144, rY=10000):
    """Plots E vs a.

    Args:
        See StabilityCalculator args.
    """
    plt.rcParams["figure.figsize"] = (10,10)

    color_vals = np.zeros([P-I+1,31], dtype=int)
    for a in range(0,31):
        for E in range(0, P-I+1):
            predictor = StabilityCalculator(I, E, N, P, R, g/52, a/100/52, h/52, rI/52, rE/52, rY/52)
            _, status, _, _ = predictor.run()
            c_value = None
            if status == 'PASS':
                c_value = 1
            elif status == 'FAIL_SS':
                c_value = 2
            elif status == 'FAIL_IC':
                c_value = 3
            elif status == 'FAIL_MENTOR':
                c_value = 4
            elif status == 'FAIL_POSITION':
                c_value = 5
            else: #FAIL_YFR
                c_value = 6
            color_vals[E,a] = c_value     
    values = [0,1,2,3,4,5,6,7]
    cmp = ListedColormap(['white','limegreen', 'lightslategray', 'chocolate', 'royalblue', 'crimson', 'magenta', 'black'])
    im = plt.imshow(color_vals, origin='lower', cmap=cmp, vmin=0, vmax=7)
    colors = [im.cmap(value) for value in values]
    patches = [ mpatches.Patch(color=colors[1], label="Success"),
            mpatches.Patch(color=colors[2], label="Steady State Failure"),
            mpatches.Patch(color=colors[3], label="Initial Condition Failure"),
            mpatches.Patch(color=colors[4], label="Mentor-Limited Crash"),
            mpatches.Patch(color=colors[5], label="Position-Limited Crash"),
            mpatches.Patch(color=colors[6], label="YFR-Limited End State")]
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.title('Stability Based on Initial Experienced Pilots and Attrition Rate')
    plt.xlabel('a')
    plt.ylabel('E0')
    plt.xticks(range(0,31,5), labels=[x/100 for x in range(0,31, 5)])
    plt.xticks(np.arange(-.5, P+0.5, 1), minor=True)
    plt.yticks(range(0,P-I+1,5))
    plt.yticks(np.arange(-.5, P-I+0.5, 1), minor=True)
    plt.grid(which='minor')

widgets.interact_manual(plot_outputs, N=widgets.IntSlider(value=25, min=0, max=100),
               P=widgets.IntSlider(value=30, min=0, max=100),
               R=widgets.IntSlider(value=250, min=0, max=1000),
               g=widgets.IntSlider(value=4, min=0, max=100),
               h=widgets.IntSlider(value=1, min=0, max=100),
               rI=widgets.IntSlider(value=144, min=0, max=1000),
               rE=widgets.IntSlider(value=144, min=0, max=1000),
               rY=widgets.IntSlider(value=10000, min=0, max=100000),
               I=widgets.IntSlider(value=10, min=0, max=100))
pass

interactive(children=(IntSlider(value=10, description='I'), IntSlider(value=25, description='N'), IntSlider(va…

### Stability Based on Initial Inexperienced Pilots and Attrition Rate

This plots the final sustainability of the system with varying values of I and a.

In [45]:
%matplotlib inline
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

def plot_predictions(E=10, N=25, P=30, R=250, g=4,
                     h=1, rI=144, rE=144, rY=10000):
    """Plots I vs a.

    Args:
        See StabilityCalculator args.
    """
    
    plt.rcParams["figure.figsize"] = (10,10)

    color_vals = np.zeros([P-E+1,31], dtype=int)
    for a in range(0,31):
        for I in range(0, P-E+1):
            predictor = StabilityCalculator(I, E, N, P, R, g/52, a/100/52, h/52, rI/52, rE/52, rY/52)
            _, status, _, _ = predictor.run()
            c_value = None
            if status == 'PASS':
                c_value = 1
            elif status == 'FAIL_SS':
                c_value = 2
            elif status == 'FAIL_IC':
                c_value = 3
            elif status == 'FAIL_MENTOR':
                c_value = 4
            elif status == 'FAIL_POSITION':
                c_value = 5
            else: #FAIL_YFR
                c_value = 6
            color_vals[I,a] = c_value

    values = [0,1,2,3,4,5,6,7]
    cmp = ListedColormap(['white','limegreen', 'lightslategray', 'chocolate', 'royalblue', 'crimson', 'magenta', 'black'])
    im = plt.imshow(color_vals, origin='lower', cmap=cmp, vmin=0, vmax=7)
    colors = [im.cmap(value) for value in values]
    patches = [ mpatches.Patch(color=colors[1], label="Success"),
            mpatches.Patch(color=colors[2], label="Steady State Failure"),
            mpatches.Patch(color=colors[3], label="Initial Condition Failure"),
            mpatches.Patch(color=colors[4], label="Mentor-Limited Crash"),
            mpatches.Patch(color=colors[5], label="Position-Limited Crash"),
            mpatches.Patch(color=colors[6], label="YFR-Limited End State")]
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.title('Stability Based on Initial Inexperienced Pilots and Attrition Rate')
    plt.xlabel('a')
    plt.ylabel('I0')
    plt.xticks(range(0,31,5), labels=[x/100 for x in range(0,31, 5)])
    plt.xticks(np.arange(-.5, P+0.5, 1), minor=True)
    plt.yticks(range(0,P-E+1,5))
    plt.yticks(np.arange(-.5, P-E+0.5, 1), minor=True)
    plt.grid(which='minor')

widgets.interact_manual(plot_predictions, N=widgets.IntSlider(value=25, min=0, max=100),
               P=widgets.IntSlider(value=30, min=0, max=100),
               R=widgets.IntSlider(value=250, min=0, max=1000),
               g=widgets.IntSlider(value=4, min=0, max=100),
               h=widgets.IntSlider(value=1, min=0, max=100),
               rI=widgets.IntSlider(value=144, min=0, max=1000),
               rE=widgets.IntSlider(value=144, min=0, max=1000),
               rY=widgets.IntSlider(value=10000, min=0, max=100000),
               E=widgets.IntSlider(value=10, min=0, max=100))
pass

interactive(children=(IntSlider(value=10, description='E'), IntSlider(value=25, description='N'), IntSlider(va…

### Stability of the System Based on Initial Experienced and Non-Operational Positions

This plots the final sustainability of the system with varying values of E and N.

In [46]:
%matplotlib inline
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

def plot_predictions(I=10, a=0.07, P=30, R=250, g=4,
                     h=1, rI=144, rE=144, rY=10000):
    """Plots E vs N.

    Args:
        See StabilityCalculator args.
    """
    
    plt.rcParams["figure.figsize"] = (10,10)

    color_vals = np.zeros([P-I+1,P+1], dtype=int)
    for N in range(0,P+1):
        for E in range(0, P-I+1):
            predictor = StabilityCalculator(I, E, N, P, R, g/52, a/52, h/52, rI/52, rE/52, rY/52)
            _, status, _, _ = predictor.run()
            c_value = None
            if status == 'PASS':
                c_value = 1
            elif status == 'FAIL_SS':
                c_value = 2
            elif status == 'FAIL_IC':
                c_value = 3
            elif status == 'FAIL_MENTOR':
                c_value = 4
            elif status == 'FAIL_POSITION':
                c_value = 5
            else: #FAIL_YFR
                c_value = 6
            color_vals[E,N] = c_value

    values = [0,1,2,3,4,5,6,7]
    cmp = ListedColormap(['white','limegreen', 'lightslategray', 'chocolate', 'royalblue', 'crimson', 'magenta', 'black'])
    im = plt.imshow(color_vals, origin='lower', cmap=cmp, vmin=0, vmax=7)
    colors = [im.cmap(value) for value in values]
    patches = [ mpatches.Patch(color=colors[1], label="Success"),
            mpatches.Patch(color=colors[2], label="Steady State Failure"),
            mpatches.Patch(color=colors[3], label="Initial Condition Failure"),
            mpatches.Patch(color=colors[4], label="Mentor-Limited Crash"),
            mpatches.Patch(color=colors[5], label="Position-Limited Crash"),
            mpatches.Patch(color=colors[6], label="YFR-Limited End State")]
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.title('Stability Based on Initial Experienced Pilots and Non-Operational Positions')
    plt.xlabel('N')
    plt.ylabel('E0')
    plt.xticks(range(0,P+1,5))
    plt.xticks(np.arange(-.5, P+0.5, 1), minor=True)
    plt.yticks(range(0,P-I+1,5))
    plt.yticks(np.arange(-.5, P-I+0.5, 1), minor=True)
    plt.grid(which='minor')

widgets.interact_manual(plot_predictions, a=widgets.FloatSlider(value=0.07, min=0, max=1, step=0.01),
               P=widgets.IntSlider(value=30, min=0, max=100),
               R=widgets.IntSlider(value=250, min=0, max=1000),
               g=widgets.IntSlider(value=4, min=0, max=100),
               h=widgets.IntSlider(value=1, min=0, max=100),
               rI=widgets.IntSlider(value=144, min=0, max=1000),
               rE=widgets.IntSlider(value=144, min=0, max=1000),
               rY=widgets.IntSlider(value=10000, min=0, max=100000),
               I=widgets.IntSlider(value=10, min=0, max=10))
pass

interactive(children=(IntSlider(value=10, description='I', max=10), FloatSlider(value=0.07, description='a', m…

### Stability of the System Based on Initial Inexperienced and Non-Operational Positions

This plots the final sustainability of the system with varying values of I and N.

In [48]:
%matplotlib inline
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

def plot_predictions(E=10, a=0.07, P=30, R=250, g=4,
                     h=1, rI=144, rE=144, rY=10000):
    """Plots I vs a.

    Args:
        See StabilityCalculator args.
    """
    
    plt.rcParams["figure.figsize"] = (10,10)
    color_vals = np.zeros([P-E+1,P+1], dtype=int)
    for N in range(0,P+1):
        for I in range(0, P-E+1):
            predictor = StabilityCalculator(I, E, N, P, R, g/52, a/52, h/52, rI/52, rE/52, rY/52)
            _, status, _, _ = predictor.run()
            c_value = None
            if status == 'PASS':
                c_value = 1
            elif status == 'FAIL_SS':
                c_value = 2f
            elif status == 'FAIL_IC':
                c_value = 3
            elif status == 'FAIL_MENTOR':
                c_value = 4
            elif status == 'FAIL_POSITION':
                c_value = 5
            else: #FAIL_YFR
                c_value = 6
            color_vals[I,N] = c_value

    values = [0,1,2,3,4,5,6,7]
    cmp = ListedColormap(['white','limegreen', 'lightslategray', 'chocolate', 'royalblue', 'crimson', 'magenta', 'black'])
    im = plt.imshow(color_vals, origin='lower', cmap=cmp, vmin=0, vmax=7)
    colors = [im.cmap(value) for value in values]
    patches = [ mpatches.Patch(color=colors[1], label="Success"),
            mpatches.Patch(color=colors[2], label="Steady State Failure"),
            mpatches.Patch(color=colors[3], label="Initial Condition Failure"),
            mpatches.Patch(color=colors[4], label="Mentor-Limited Crash"),
            mpatches.Patch(color=colors[5], label="Position-Limited Crash"),
            mpatches.Patch(color=colors[6], label="YFR-Limited End State")]
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.title('Stability Based on Initial Inexperienced Pilots and Non-Operational Positions')
    plt.xlabel('N')
    plt.ylabel('I0')
    plt.xticks(range(0,P+1,5))
    plt.xticks(np.arange(-.5, P+0.5, 1), minor=True)
    plt.yticks(range(0,P-E+1,5))
    plt.yticks(np.arange(-.5, P-E+0.5, 1), minor=True)
    plt.grid(which='minor')

widgets.interact_manual(plot_predictions, a=widgets.FloatSlider(value=0.07, min=0, max=1, step=0.01),
               P=widgets.IntSlider(value=30, min=0, max=100),
               R=widgets.IntSlider(value=250, min=0, max=1000),
               g=widgets.IntSlider(value=4, min=0, max=100),
               h=widgets.IntSlider(value=1, min=0, max=100),
               rI=widgets.IntSlider(value=144, min=0, max=1000),
               rE=widgets.IntSlider(value=144, min=0, max=1000),
               rY=widgets.IntSlider(value=10000, min=0, max=100000),
               e=widgets.IntSlider(value=10, min=0, max=100))
pass

interactive(children=(IntSlider(value=10, description='E', max=30, min=-10), FloatSlider(value=0.07, descripti…

### Stability of the System Depending on Attrition Rate and Non-Operational Positions

This plots the final sustainability of the system with varying values of N and a.

In [49]:
%matplotlib inline
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt

import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

def plot_predictions(E=10, I=10, P=30, R=250, g=4,
                     h=1, rI=144, rE=144, rY=10000):
    """Plots a vs N.

    Args:
        See StabilityCalculator args.
    """
    
    plt.rcParams["figure.figsize"] = (10,10)

    color_vals = np.zeros([31,P+1], dtype=int)
    for N in range(0,P+1):
        for a in range(0, 31):
            predictor = StabilityCalculator(I, E, N, P, R, g/52, a/100/52, h/52, rI/52, rE/52, rY/52)
            _, status, _, _ = predictor.run()
            c_value = None
            if status == 'PASS':
                c_value = 1
            elif status == 'FAIL_SS':
                c_value = 2
            elif status == 'FAIL_IC':
                c_value = 3
            elif status == 'FAIL_MENTOR':
                c_value = 4
            elif status == 'FAIL_POSITION':
                c_value = 5
            else: #FAIL_YFR
                c_value = 6
            color_vals[a,N] = c_value

    values = [0,1,2,3,4,5,6,7]
    cmp = ListedColormap(['white','limegreen', 'lightslategray', 'chocolate', 'royalblue', 'crimson', 'magenta', 'black'])
    im = plt.imshow(color_vals, origin='lower', cmap=cmp, vmin=0, vmax=7)
    colors = [im.cmap(value) for value in values]
    patches = [ mpatches.Patch(color=colors[1], label="Success"),
            mpatches.Patch(color=colors[2], label="Steady State Failure"),
            mpatches.Patch(color=colors[3], label="Initial Condition Failure"),
            mpatches.Patch(color=colors[4], label="Mentor-Limited Crash"),
            mpatches.Patch(color=colors[5], label="Position-Limited Crash"),
            mpatches.Patch(color=colors[6], label="YFR-Limited End State")]
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.title('Stability Based on Non-Operational Positions and Attrition Rate')
    plt.xlabel('N')
    plt.ylabel('a')
    plt.xticks(range(0,P+1,5))
    plt.xticks(np.arange(-.5, P+0.5, 1), minor=True)
    plt.yticks(range(0,31,5), labels=[x/100 for x in range(0,31,5)])
    plt.yticks(np.arange(-.5, 31, 1), minor=True)
    plt.grid(which='minor')
  
widgets.interact_manual(plot_predictions, I=widgets.IntSlider(value=10, min=0, max=100),
               E=widgets.IntSlider(value=10, min=0, max=100),
               P=widgets.IntSlider(value=30, min=0, max=100),
               R=widgets.IntSlider(value=250, min=0, max=1000),
               g=widgets.IntSlider(value=4, min=0, max=100),
               h=widgets.IntSlider(value=1, min=0, max=100),
               rI=widgets.IntSlider(value=144, min=0, max=1000),
               rE=widgets.IntSlider(value=144, min=0, max=1000),
               rY=widgets.IntSlider(value=10000, min=0, max=100000))
pass

interactive(children=(IntSlider(value=10, description='E'), IntSlider(value=10, description='I'), IntSlider(va…